# HMM Map-Matching (Newson & Krumm, 2009) Implementation

## Algorithm
<div align="center">
  <img src="img/img1.png" width="300"/>
  <img src="img/img2.png" width="300"/>
</div>

*Variable Definitions:*

- $z_t:$ location measurement (GPS observation) at time step $t$
- $r_{t,i}:$ candidate road segment for observation $z_t$
- $x_{t,i}:$ the point on road segment $r_{t,i}$ nearest to the GPS observation $z_t$.
- $N_t:$ total number of candidate road segments for GPS observation $z_t$
- $\left\lVert a - b \right\rVert_{\mathrm{great\ circle}}:$ shortest distance between $a$ and $b$ on a sphere
- $\left\lVert a - b \right\rVert_{\mathrm{route}}:$ shortest distance along the road network between $a$ and $b$

*Emission Probabilities:* Gives the likelihood that the measurement $z_t$ would be observed if the vehicle were actually on
road segment $r_i$
$$
p(z_t \mid r_i) = \frac{1}{\sqrt{2\pi}\,\sigma_z}
\exp\!\left(
   -\tfrac{1}{2}
   \left(
     \frac{\lVert z_t - x_{t,i} \rVert_{\text{great circle}}}{\sigma_z}
   \right)^{\!2}
\right)
$$

So we have $e_t =\begin{bmatrix}
\log p(z_t \mid r_{t,1}) \\
\log p(z_t \mid r_{t,2}) \\
\vdots \\
\log p(z_t \mid r_{t,N_t})
\end{bmatrix} \in \mathbb{R}^{N_t}$

*Transition Probabilities:* Gives the likelihood that a traveler moved from $r_{t,i}$ to $r_{{t+1},j}$ during the time between the two GPS samples

$$p(r_{{t+1},j}|r_{t,i}) = \frac{1}{\beta} \, e^{-d_t/\beta}$$

$$d_t = \left| \,\left\lVert z_{t+1} - z_{t} \right\rVert_{\mathrm{great\ circle}}
      - \left\lVert x_{t+1,j^{*}} - x_{t,i^{*}} \right\rVert_{\mathrm{route}} \,\right|$$

So we have $A^{(t)}= \begin{bmatrix}
\log p(r_{t+1,1} \mid r_{t,1}) & \log p(r_{t+1,2} \mid r_{t,1}) & \cdots & \log p(r_{t+1,N_{t+1}} \mid r_{t,1}) \\
\log p(r_{t+1,1} \mid r_{t,2}) & \log p(r_{t+1,2} \mid r_{t,2}) & \cdots & \log p(r_{t+1,N_{t+1}} \mid r_{t,2}) \\
\vdots & \vdots & \ddots & \vdots \\
\log p(r_{t+1,1} \mid r_{t,N_t}) & \log p(r_{t+1,2} \mid r_{t,N_t}) & \cdots & \log p(r_{t+1,N_{t+1}} \mid r_{t,N_t})
\end{bmatrix}
\in \mathbb{R}^{N_t \times N_{t+1}}\in \mathbb{R}^{N_t \times N_{t+1}}, \quad
\bigl[A^{(t)}\bigr]_{ij} = \log p\!\bigl(r_{t+1,j} \mid r_{t,i}\bigr)$

*Viterbi Algorithm (Dynamic programming):*

Base case:

$V_1(j) = [\mathbf{e}_1]_j, \qquad B_1(j) = -1$

Recursion:

$V_t(j) = [\mathbf{e}_t]_j +
\max_{i \in \{1,\dots,N_{t-1}\}}
\left\{ V_{t-1}(i) + [\mathbf{A}^{(t-1)}]_{ij} \right\}$

$B_t(j) =
\arg\max_{i}
\left\{ V_{t-1}(i) + [\mathbf{A}^{(t-1)}]_{ij} \right\}$

$B_t(j)$ is the index of the previous candidate at time $t-1$ that gave the highest score when we decide to go into candidate $j$ at time $t$


Termination:

$j^{*} = \arg\max_{j} V_T(j)$

Output:

$\hat{r}_T = r_{T,j^{*}}, \qquad
\hat{r}_t = r_{t, B_{t+1}(\hat{r}_{t+1})}
\quad \text{for } t = T-1,\dots,1$

## Updated: some paper use t-distribition













In [15]:
!pip3 install geopandas shapely pyproj

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import json
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd

In [2]:
### Get 10 samples from the trip
with open("trip5000.json", "r") as f:
    data = json.load(f)

print("Top-level type:", type(data))
print("Keys:", list(data.keys()))

subset = {
    "type": data["type"],
    "name": data.get("name"),
    "features": data["features"][:10]
}

with open("trip10.json", "w") as f:
    json.dump(subset, f, indent=2)

Top-level type: <class 'dict'>
Keys: ['type', 'name', 'features']


In [3]:
### Load the 10 samples and the road network
trip = gpd.read_file("trip10.json")
roads = gpd.read_file("geobase/geobase_mtl.shp")

In [4]:
### Add timestamps spaced evenly to each point
trip["trip_id"] = trip.index

def linestring_to_points(row):
    coords = list(row.geometry.coords)
    n = len(coords)
    start = pd.to_datetime(row.get("start"))
    stop  = pd.to_datetime(row.get("stop"))

    if pd.isnull(start) or pd.isnull(stop):
        ts = [None] * n
    else:
        ts = pd.date_range(start, stop, periods=n)
    return pd.DataFrame({
        "geometry": [Point(xy) for xy in coords],
        "t": ts,
        "trip_id": row.trip_id
    })

pts = pd.concat(
    [linestring_to_points(r) for _, r in trip.iterrows()],
    ignore_index=True
)

pts = gpd.GeoDataFrame(pts, geometry="geometry", crs=trip.crs)
print(pts.groupby("trip_id").size())

trip_id
0     408
1      77
2     857
3     355
4     714
5     110
6    1184
7    1147
8    3563
9    1431
dtype: int64


In [5]:
### EPSG:4326 to meters
pts_m = pts.to_crs("EPSG:26918")

*Parameter Estimation:*
There are two parameters to be estimated.

1) $\sigma_z$ : the standard deviation of Gaussian GPS noise. In Newson \& Krumm(2009), $\sigma_z$ is estimated using the median absolute deviation:
$$
\sigma_z = 1.4826 \, \mathrm{median}_t\!\left( \bigl\lVert z_t - x_{t,i^*} \bigr\rVert_{\mathrm{great\;circle}} \right)
$$
where $x_{t,i^*}$ is the point on the true matched road segment closest to $z_t$.

2) $\beta$: the difference between the route distance and the great-circle distance.

$$
\beta = \frac{1}{\ln(2)} \, \mathrm{median}_t \!\left(
  \bigl\lVert z_t - z_{t+1} \bigr\rVert_{\mathrm{great\;circle}}
  \;-\;
  \bigl\lVert x_{t,i^*} - x_{t+1,j^*} \bigr\rVert_{\mathrm{route}}
\right)
\tag{6}
$$

"A larger value of $\sigma_z$, which measures noise in the location measurements, represents less trust in the location measurements."

"A larger value of $\beta$, which measures the difference between great circle distances and route distances, represents more tolerance of non-direct routes."

"In our work, we estimate these two parameters directly from the data. An alternative would be to find the values of $\sigma_z$ and $\beta$ that optimize performance of the algorithm. We leave this for future work."

## Preprocessing
In Newson \& Krumm(2009),remove points that are within $2\sigma_z$ of the previous included point. The justification for this step is that until we see a point that is at least $2\sigma_z$ away from its temporal predecessor, our confidence is low that the apparent movement is due to actual vehicle movement and not noise. (They assume 1Hz sampling and typical car speed)


NEW ADD: Additionally, trips consisting of two or fewer data points were excluded as they are not suitable for map-matching.




In [6]:
sigma_z = 3.0
keep = []
last_kept = None

for i, p in pts_m.iterrows():
    if last_kept is None:
        keep.append(True)
        last_kept = p.geometry
    else:
        d = last_kept.distance(p.geometry)
        if d >= sigma_z:
            keep.append(True)
            last_kept = p.geometry
        else:
            keep.append(False)

pts_m = pts_m[keep].reset_index(drop=True)

counts_after = pts_m.groupby("trip_id").size()
print(counts_after)

trip_id
0     248
1      69
2     761
3     332
4     693
5     105
6    1143
7    1093
8    3408
9    1269
dtype: int64


# HMM Breaks
A break means there are no transitions with non-zero probability from any candidate road segment at time $t$ to any candidate road segment at time $t+1$. (Every candidate pair gives near-zero transition probability)
In Newson & Krumm, 2009, they explained three main reasons these breaks occur:

1)  To reduce computation, the algorithm ignores road segments farther than 200 m from a GPS point. If a point has no nearby roads that time step ends up with zero candidates.
2)  If the shortest road route is much longer  (at least 2 kilometers more than the straight line). Such a big gap means the vehicle would have had to take an unnecessarily long or looping detour. The algorithm then sets the transition probability for that connection to zero and stops considering it, so it doesn’t waste effort on obviously unrealistic paths.
3)


# Some Preprocessing Steps From Other Papers
50-meter radius: pgmamtching use 50 meter radis, They build a main road network backbone and then decide which roads are realistically usable by bikes — they purposely downweight or exclude main car roads if there’s a safe parallel cycle path.


In the candidate road selection process, if there are only two or fewer data points for which we can find at least one road segment within 50 meters, these trips are removed.





In [14]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


MAX_BREAK = 180   # seconds
MAX_SPEED = 50    # m/s (~180 km/h)
SIGMA_JUMP = 200  # meters

def transition_possible(p1, p2):
    if pd.isnull(p1['t']) or pd.isnull(p2['t']):
        return True
    dt = (p2['t'] - p1['t']).total_seconds()
    if dt <= 0:
        return False
    dist = p1.geometry.distance(p2.geometry)
    speed = dist / dt

    return dist < SIGMA_JUMP and speed < MAX_SPEED


def heal_trip(points):

    trips = []
    start_idx = 0
    i = 0
    while i < len(points) - 1:
        if transition_possible(points.iloc[i], points.iloc[i+1]):
            i += 1
            continue

        left = i
        right = i + 1
        while left >= start_idx and right < len(points):

            t_left = points.iloc[left].t
            t_right = points.iloc[right].t
            span = abs((t_right - t_left).total_seconds()) if pd.notnull(t_left) and pd.notnull(t_right) else 0

            if left > start_idx and right + 1 < len(points):
                if transition_possible(points.iloc[left - 1], points.iloc[right + 1]):
                    # heal: drop points [left..right]
                    healed = pd.concat([points.iloc[start_idx:left],
                                        points.iloc[right + 1:]])
                    return [healed.reset_index(drop=True)]

            if span > MAX_BREAK:
                trips.append(points.iloc[start_idx:left + 1].copy())

                points = points.iloc[right:].reset_index(drop=True)
                start_idx = 0
                i = 0
                break

            left -= 1
            right += 1
        else:
            break


    if not trips:
        return [points.reset_index(drop=True)]
    trips.append(points.reset_index(drop=True))
    return trips


healed_trips = []
next_id = 0

for tid, group in pts_m.groupby("trip_id"):

    group = group.sort_values("t").reset_index(drop=True)
    new_segments = heal_trip(group)
    for seg in new_segments:
        seg["trip_id"] = next_id
        next_id += 1
        healed_trips.append(seg)

pts_final = gpd.GeoDataFrame(pd.concat(healed_trips, ignore_index=True),
                             geometry="geometry", crs=pts_m.crs)

print("Trips after healing:", pts_final["trip_id"].nunique())
print(pts_final.groupby("trip_id").size())

Trips after healing: 10
trip_id
0     248
1      69
2     761
3     332
4     693
5     105
6    1141
7    1093
8    3408
9    1269
dtype: int64
